In [17]:
import numpy as np
import pandas as pd
from datamatch import (
    ThresholdMatcher,
    DissimilarFilter,
    NonOverlappingFilter,
    ColumnsIndex,
    JaroWinklerSimilarity,
    MaxScorer,
    SimSumScorer,
    AlterScorer,
    MultiIndex,
)
from datavalid.spinner import Spinner

In [18]:
def read_personnel():
    dfa = pd.read_csv("./../../../data/GA/match/personnel_georgia_dropped_na_left_date.csv")
    dfa = dfa.rename(columns={"year_of_birth": "birth_year"})
    dfb = pd.read_csv("./../../../data/FL/match/personnel_florida_dropped_na_left_dates.csv")

    dfa = dfa[["first_name", "last_name", "middle_name", "race", "sex", "birth_year", "state", "uid"]]

    dfb = dfb[["first_name", "last_name", "middle_name", "race", "sex", "birth_year", "state", "uid"]]

    df = pd.concat([dfa, dfb])
    return df


def read_events():
    dfa = pd.read_csv("./../../../data/FL/match/events_florida_dropped_na_left_dates.csv")

    dfa = dfa[["uid", "event_uid", "event_type", "event_date", "agency", "state"]]
    dfb = pd.read_csv("./../../../data/GA/match/events_georgia_dropped_na_left_date.csv")

    dfb = dfb[["uid", "event_uid", "event_type", "event_date", "agency", "state"]]

    df = pd.concat([dfa, dfb])

    df = df[~((df.event_date.fillna("") == ""))]

    df = df.rename(columns={"event_type": "kind"})

    dates = df.event_date.str.extract(r"(\w+)\/(\w+)\/(\w+)")

    df.loc[:, "day"] = dates[1].fillna("")
    df.loc[:, "month"] = dates[0].fillna("")
    df.loc[:, "year"] = dates[2].fillna("")

    df = df[~((df.day == ""))]

    df.loc[:, "day"] = df.day.astype(int)

    df.loc[:, "month"] = df.month.astype(int)

    df.loc[:, "year"] = df.year.astype(int)

    df = df.drop(columns=["event_date"])

    return df, dfa, dfb

In [19]:
events_df, dfa, dfb = read_events()
events_df

## check if ledt_date in kind 

,uid,event_uid,kind,agency,state,day,month,year
0,b4cbffe2a853f9474ec550c2ccbaff90,fa75a27a3d71b1ee5d71eece2a07ea77,hire_date,Oak Hill Police Department,FL,18,1,2000
1,b4cbffe2a853f9474ec550c2ccbaff90,d7fb7ea1856440187493093b1b8e1097,hire_date,Oak Hill Police Department,FL,5,9,2001
2,dd83af4c6302340ce7ccfc9ac46a0d9f,f1e316950e7483dd2060ba6defb46d77,hire_date,Orange County Sheriff's Office,FL,15,10,1992
3,dd83af4c6302340ce7ccfc9ac46a0d9f,473c370586c14e716e73ea47024be8c6,hire_date,Orange County Sheriff's Office,FL,18,2,1992
4,417a46bf049b6a59cc7ed0809336f7bb,732a44c858d555f907cdedb23fb2c002,hire_date,Pahokee Police Department,FL,4,9,1998
...,...,...,...,...,...,...,...,...
153093,695120a6b3ed96231064d9b7ba26c840,fa11a723a01afe11c68084dde42f3ea5,left_date,G1313 SCREVEN COUNTY SHERIFFS OFFICE,GA,1,7,2022
153094,8ecfb825cfa0d3c5cd4108d51b03d602,a396cf8471476a5b674bdd16accbcc45,left_date,G1587 NEWINGTON POLICE DEPARTMENT,GA,19,1,2020
153095,8ecfb825cfa0d3c5cd4108d51b03d602,6b788563c881be44c0946b518cc39832,left_date,G1587 NEWINGTON POLICE DEPARTMENT,GA,1,1,2021
153096,55526e61d2cb0314c9e8bfd2c5455b5d,928fd11d5a5469467ea622936a774abb,left_date,G1505 ATLANTA POLICE DEPARTMENT,GA,8,1,2020


In [20]:
dfa

,uid,event_uid,event_type,event_date,agency,state
0,b4cbffe2a853f9474ec550c2ccbaff90,fa75a27a3d71b1ee5d71eece2a07ea77,hire_date,01/18/2000,Oak Hill Police Department,FL
1,b4cbffe2a853f9474ec550c2ccbaff90,d7fb7ea1856440187493093b1b8e1097,hire_date,09/05/2001,Oak Hill Police Department,FL
2,dd83af4c6302340ce7ccfc9ac46a0d9f,f1e316950e7483dd2060ba6defb46d77,hire_date,10/15/1992,Orange County Sheriff's Office,FL
3,dd83af4c6302340ce7ccfc9ac46a0d9f,473c370586c14e716e73ea47024be8c6,hire_date,02/18/1992,Orange County Sheriff's Office,FL
4,417a46bf049b6a59cc7ed0809336f7bb,732a44c858d555f907cdedb23fb2c002,hire_date,09/04/1998,Pahokee Police Department,FL
...,...,...,...,...,...,...
1062949,fa9193c2b02a434a7b41da5921b98f4c,a10198c28e3c0ef8c1abae114cd572c5,left_date,10/14/2022,Fort Lauderdale Police Department,FL
1062950,00c79e0a486150b6076b319d6529aafb,572b228479f0f5cc45f96492b07c664b,left_date,04/03/2023,New Port Richey Police Department,FL
1062951,8e0ed14abc2336c1dc0c5b30357bd71f,2cb05b597b93e3d5e9035eeee9ee449e,left_date,03/24/2023,Lady Lake Police Department,FL
1062952,4c874798027c548e7d0d2fa6e6e9da7f,70e74859544c5ca96bd600377b2be8ad,left_date,03/23/2023,Blackwater River Correctional Facility,FL


In [21]:
dfb

,uid,event_uid,event_type,event_date,agency,state
0,1ecd52b930590412dbceaceaa97a1846,6e4fa0a0c333c1f05bf54cc1db5df6f9,hire_date,04/19/1999,G1161 CHATHAM COUNTY SHERIFFS OFFICE,GA
1,a8d978d99f97b0a827a097556bcc13ed,ab94d859ff4dcd0d3663960e25ee2ff4,hire_date,10/09/2000,G1244 SAVANNAH POLICE DEPARTMENT,GA
2,50c573910e56846b39df32389989a8e0,52755366a16785d593b0bfd19d76049c,hire_date,03/10/2001,G1177 POOLER POLICE DEPARTMENT,GA
3,9ed903f0c60ed2b59cf9732bddca10b9,84a8d62ed4a904127dff2abf52b82a32,hire_date,04/09/2009,G1423 JENKINS COUNTY SHERIFFS OFFICE,GA
4,9ed903f0c60ed2b59cf9732bddca10b9,9685d48d77470db86070f89fd19460dd,hire_date,01/01/2012,G1423 JENKINS COUNTY SHERIFFS OFFICE,GA
...,...,...,...,...,...,...
153093,695120a6b3ed96231064d9b7ba26c840,fa11a723a01afe11c68084dde42f3ea5,left_date,07/01/2022,G1313 SCREVEN COUNTY SHERIFFS OFFICE,GA
153094,8ecfb825cfa0d3c5cd4108d51b03d602,a396cf8471476a5b674bdd16accbcc45,left_date,01/19/2020,G1587 NEWINGTON POLICE DEPARTMENT,GA
153095,8ecfb825cfa0d3c5cd4108d51b03d602,6b788563c881be44c0946b518cc39832,left_date,01/01/2021,G1587 NEWINGTON POLICE DEPARTMENT,GA
153096,55526e61d2cb0314c9e8bfd2c5455b5d,928fd11d5a5469467ea622936a774abb,left_date,01/08/2020,G1505 ATLANTA POLICE DEPARTMENT,GA


In [22]:
per_df = read_personnel()
per_df

,first_name,last_name,middle_name,race,sex,birth_year,state,uid
0,damon,aanerud,h,NaN,,1972.0,GA,1ecd52b930590412dbceaceaa97a1846
1,damon,aanerud,h,NaN,,1972.0,GA,a8d978d99f97b0a827a097556bcc13ed
2,damon,aanerud,h,NaN,,1972.0,GA,50c573910e56846b39df32389989a8e0
3,wesley,aaron,alan,NaN,,1977.0,GA,9ed903f0c60ed2b59cf9732bddca10b9
4,wesley,aaron,alan,NaN,,1977.0,GA,9ed903f0c60ed2b59cf9732bddca10b9
...,...,...,...,...,...,...,...,...
550248,eliezer,rojas,leonardo,hispanic,male,1984.0,FL,00c79e0a486150b6076b319d6529aafb
550249,nicholas,lesses,NaN,white,male,2002.0,FL,8e0ed14abc2336c1dc0c5b30357bd71f
550250,austin,smith,tyler,white,male,1995.0,FL,4c874798027c548e7d0d2fa6e6e9da7f
550251,devon,williams,NaN,black,male,2000.0,FL,13539545d26fc06603eee4975dc187b6


In [23]:

def combine_date_columns(
    df: pd.DataFrame, year_col: str, month_col: str, day_col: str
) -> pd.Series:
    """Combines date columns into a single column

    Args:
        df (pd.DataFrame):
            the frame to process
        year_col (str):
            year column
        month_col (str):
            month column
        day_col (str):
            day column

    Returns:
        the combined datetime series
    """
    dates = df[[year_col, month_col, day_col]]
    dates.columns = ["year", "month", "day"]
    return pd.to_datetime(dates, errors="coerce")


def combine_datetime_columns(
    df: pd.DataFrame, year_col: str, month_col: str, day_col: str, time_col: str
) -> pd.Series:
    """Combines datetime columns into a single column

    Args:
        df (pd.DataFrame):
            the frame to process
        year_col (str):
            year column
        month_col (str):
            month column
        day_col (str):
            day column
        time_col (str):
            time column

    Returns:
        the combined datetime series
    """
    time_frame = df[time_col].str.split(":", expand=True)
    dates = pd.concat([df[[year_col, month_col, day_col]], time_frame], axis=1)
    dates.columns = ["year", "month", "day", "hour", "minute"]
    return pd.to_datetime(dates)


def discard_rows(
    events: pd.DataFrame, bool_index: pd.Series, desc: str, reset_index: bool = False
) -> pd.DataFrame:
    before = events.shape[0]
    events = events[bool_index]
    if reset_index:
        events = events.reset_index(drop=True)
    after = events.shape[0]
    if before > after:
        print(
            "discarded %d %s (%.1f%%)"
            % (before - after, desc, (before - after) / before * 100)
        )
    return events


def assign_min_col(events: pd.DataFrame, per: pd.DataFrame, col: str):
    # Group by 'uid' and calculate the minimum
    min_series = events.groupby(level='uid')[col].min()
    min_dict = min_series.to_dict()
    # Map minimum values to the 'per' DataFrame
    per.loc[:, "min_" + col] = per.index.map(lambda x: min_dict.get(x, np.NaN))


def assign_max_col(events: pd.DataFrame, per: pd.DataFrame, col: str):
    # Group by 'uid' and calculate the maximum
    max_series = events.groupby(level='uid')[col].max()
    max_dict = max_series.to_dict()
    # Map maximum values to the 'per' DataFrame
    per.loc[:, "max_" + col] = per.index.map(lambda x: max_dict.get(x, np.NaN))


def cross_match_officers_between_agencies(personnel, events):
    events = discard_rows(
        events, events.uid.notna(), "events with empty uid column", reset_index=True
    )
    events = discard_rows(
        events, events.day.notna(), "events with empty day column", reset_index=True
    )
    events = discard_rows(
        events, events.day <= 31, "events with impossible day column", reset_index=True
    )
    for col in ["year", "month", "day"]:
        events.loc[:, col] = events[col].astype(int)
    events.loc[:, "date"] = combine_date_columns(events, "year", "month", "day")
    events = discard_rows(
        events, events.date.notna(), "events with empty date", reset_index=True
    )

    # Convert 'date' to datetime format
    events['date'] = pd.to_datetime(events['date'], errors='coerce')

    # Create 'timestamp' from 'date' and convert it to integers
    events.loc[:, "timestamp"] = events["date"].map(lambda x: x.timestamp())
    events['timestamp'] = pd.to_numeric(events['timestamp'], errors='coerce')


    per = personnel[["uid", "first_name", "last_name", "race", "sex", "birth_year", "state"]]

    per.loc[:, "birth_year"] = per.birth_year.astype(str)
    
    per = per.drop_duplicates(subset=["uid"])

    per = discard_rows(
        per,
        per.first_name.notna() & per.last_name.notna(),
        "officers without either first name or last name",
        reset_index=True,
    )
    per.loc[:, "fc"] = per.first_name.map(lambda x: x[:5])
    per.loc[:, "lc"] = per.last_name.map(lambda x: x[:5])
    agency_dict = (
        events.loc[:, ["uid", "agency"]]
        .drop_duplicates()
        .set_index("uid")
        .agency.to_dict()
    )
    per.loc[:, "agency"] = per.uid.map(lambda x: agency_dict.get(x, ""))
    per = discard_rows(
        per, per.agency != "", "officers not linked to any event", reset_index=True
    )


    per = per.set_index("uid")

    # aggregating min/max date
    events = events.set_index(["uid", "event_uid"])
    assign_min_col(events, per, "date")
    assign_max_col(events, per, "date")
    assign_min_col(events, per, "timestamp")
    assign_max_col(events, per, "timestamp")
    per = discard_rows(per, per.min_date.notna(), "officers with no event")

    # concatenate first name and last name to get a series of full names
    full_names = per.first_name.str.cat(per.last_name, sep=" ")
    # filter down the full names to only those that are common
    # common_names_sr = pd.Series([x for x in full_names])
    
    
    ##### check race and sex cols compatbility 

    excel_path = "../../../data/FL/match/cross_agency_officers.xlsx"
    matcher = ThresholdMatcher(
        index=MultiIndex(
            [
                ColumnsIndex(["fc", "lc"]),
            ]
        ),
        scorer=MaxScorer(
            [
                AlterScorer(
                    # calculate similarity score (0-1) based on name similarity
                    scorer=SimSumScorer(
                        {
                            "first_name": JaroWinklerSimilarity(),
                            "last_name": JaroWinklerSimilarity(),
                            "birth_year": JaroWinklerSimilarity(),
                        }
                    ),
                    # but for pairs that have the same name and their name is common
                    values=full_names,
                    # give a penalty of -.2 which is enough to eliminate them
                    alter=lambda score: score - 0.2,
                ),
            ]
        ),
        dfa=per,
        filters=[
            # don't match officers who belong in the same agency
            DissimilarFilter("state"),
            # don't match officers who appear in overlapping time ranges
            NonOverlappingFilter("min_timestamp", "max_timestamp"),
        ],
        show_progress=True,
    )
    decision = .5
    with Spinner("saving matched clusters to Excel file"):
        matcher.save_clusters_to_excel(excel_path, decision, lower_bound=decision)
    clusters = matcher.get_index_clusters_within_thresholds(decision)
    print("saved %d clusters to %s" % (len(clusters), excel_path))

    return clusters, per[["max_timestamp", "agency", "race", "sex", "birth_year", "state"]]


In [24]:
matches = cross_match_officers_between_agencies(per_df, events_df)

discarded 183 events with empty date (0.0%)


/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_74228/3828131139.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per.loc[:, "birth_year"] = per.birth_year.astype(str)


discarded 40 officers without either first name or last name (0.0%)


scoring pairs: 113592it [00:46, 2427.61it/s]

⠋ saving matched clusters to Excel file

saved 11527 clusters to ../../../data/FL/match/cross_agency_officers.xlsxfile saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving matched clusters to Excel file saving mat